# Data prepation with daily aggregation
The observation period is split into 8 periods and purchase history is aggregated by client and period and then flattened into a two-dimentional data set.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Client data

In [3]:
clients = pd.read_csv('clients2.csv',
                     index_col='client_id',
                     parse_dates=['first_issue_date', 'first_redeem_date'],
                     na_values={'gender': 'U'})\
    .drop(columns='client_id.1')
clients

,first_issue_date,first_redeem_date,age,gender
client_id,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,NaN
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,NaN
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,NaN
...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,NaN
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F


# Age

There are negative values and values above 100 (as high as 1852).

Calculate mean age using clients within reasonable age range:

In [4]:
mean_age = clients.loc[(clients.age < 90) & (clients.age > 10), 'age'].mean()
mean_age

46.37953291808248

In [5]:
clients.loc[(clients.age > 90) | (clients.age < 10), 'age'] = mean_age

# Gender

In [6]:
clients = pd.get_dummies(clients, columns=['gender'])
clients

,first_issue_date,first_redeem_date,age,gender_F,gender_M
client_id,,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45.0,0,0
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72.0,1,0
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83.0,0,0
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33.0,1,0
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73.0,0,0
...,...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35.0,1,0
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69.0,0,0
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73.0,1,0


# Issue and redeem dates

## Clients that have not redeemed

In [7]:
clients['no_redeem'] = clients.first_redeem_date.isna().astype('int')

In [8]:
clients.isna().sum()

first_issue_date         0
first_redeem_date    17546
age                      0
gender_F                 0
gender_M                 0
no_redeem                0
dtype: int64

In [9]:
clients.loc[clients.no_redeem == 1, :]

,first_issue_date,first_redeem_date,age,gender_F,gender_M,no_redeem
client_id,,,,,,
00068fd5dc,2018-12-15 11:14:26,NaT,72.000000,1,0,1
0006fca4bf,2019-02-20 10:47:30,NaT,57.000000,0,0,1
0009e6bafa,2018-10-01 18:40:29,NaT,32.000000,0,0,1
000d599743,2018-08-29 09:08:39,NaT,46.379533,0,0,1
00140e5d34,2019-03-07 13:10:31,NaT,50.000000,1,0,1
...,...,...,...,...,...,...
ffe2d2bdbc,2018-02-16 12:06:15,NaT,82.000000,0,0,1
ffe4b6aa1a,2019-01-19 19:01:44,NaT,56.000000,0,0,1
fff2b6bf63,2018-10-18 19:01:34,NaT,46.000000,0,0,1


## Use the last redeem date to fill missing values

In [10]:
last_redeem = clients.first_redeem_date.max(); last_redeem

Timestamp('2019-11-20 01:14:10')

In [11]:
clients.fillna(last_redeem, inplace=True)

In [12]:
clients.isna().sum()

first_issue_date     0
first_redeem_date    0
age                  0
gender_F             0
gender_M             0
no_redeem            0
dtype: int64

In [13]:
clients['first_issue_date'] = (clients['first_issue_date'] - pd.Timestamp("1970-01-01")).dt.days
clients['first_redeem_date'] = (clients['first_redeem_date'] - pd.Timestamp("1970-01-01")).dt.days
clients

,first_issue_date,first_redeem_date,age,gender_F,gender_M,no_redeem
client_id,,,,,,
000012768d,17383,17535,45.0,0,0,0
000036f903,17266,17279,72.0,1,0,0
00010925a5,17736,17788,83.0,0,0,0
0001f552b0,17347,17771,33.0,1,0,0
00020e7b18,17497,17541,73.0,0,0,0
...,...,...,...,...,...,...
fffe0abb97,17497,17573,35.0,1,0,0
fffe0ed719,17424,17512,69.0,0,0,0
fffea1204c,17562,17602,73.0,1,0,0


### Scaling

In [14]:
contin_vars = ['first_issue_date', 'first_redeem_date', 'age']

In [15]:
scaler = StandardScaler()

In [16]:
scaled = scaler.fit_transform(clients[contin_vars])

In [17]:
clients[contin_vars] = pd.DataFrame(scaled, columns=contin_vars, index=clients.index)
clients

,first_issue_date,first_redeem_date,age,gender_F,gender_M,no_redeem
client_id,,,,,,
000012768d,-0.769796,-0.855610,-0.087012,0,0,0
000036f903,-1.344307,-1.861964,1.615538,1,0,0
00010925a5,0.963560,0.138951,2.309169,0,0,0
0001f552b0,-0.946568,0.072123,-0.843701,1,0,0
00020e7b18,-0.210015,-0.832023,1.678595,0,0,0
...,...,...,...,...,...,...
fffe0abb97,-0.210015,-0.706229,-0.717586,1,0,0
fffe0ed719,-0.568471,-0.946024,1.426365,0,0,0
fffea1204c,0.109158,-0.592228,1.678595,1,0,0


In [18]:
clients.to_csv('clients2_modified.csv')

# Purchase data aggregation

In [2]:
test_purch = pd.read_csv('test_purch.csv', parse_dates=['transaction_datetime'])

In [3]:
train_purch = pd.read_csv('train_purch.csv', parse_dates=['transaction_datetime'])
train_purch

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15998947,fffeca6d22,a0bb11a968,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,de437430e2,1.0,19.0,NaN
15998948,fffeca6d22,a0bb11a968,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,ad8fee4200,1.0,45.0,NaN
15998949,fffeca6d22,3b876a5532,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,ad8fee4200,2.0,94.0,NaN
15998950,fffeca6d22,3b876a5532,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,230dbb0c16,1.0,50.0,NaN


In [4]:
test_purch = test_purch\
    .groupby(['client_id','transaction_datetime'],
            as_index=False)\
    .agg({'regular_points_received': 'mean',
          'express_points_received': 'mean',
          'regular_points_spent': 'mean', 
          'express_points_spent': 'mean', 
          'purchase_sum': 'mean',
         'product_id': 'count',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
test_purch.rename(columns = {'product_id': 'n_products'}, inplace=True)
test_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,5,7.0,303.0,0.0
1,00010925a5,2018-12-01 09:17:18,0.8,0.0,0.0,0.0,132.0,3,2.0,132.0,0.0
2,00010925a5,2018-12-03 07:52:46,0.7,0.0,0.0,0.0,149.0,3,3.0,150.0,0.0
3,00010925a5,2018-12-06 09:59:19,1.7,0.0,0.0,0.0,349.0,6,7.0,349.0,0.0
4,00010925a5,2018-12-17 12:51:26,5.8,0.0,0.0,0.0,581.0,8,8.0,582.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1209945,fffff6ce77,2019-02-28 12:56:31,1.8,0.0,0.0,0.0,446.0,9,10.0,446.0,0.0
1209946,fffff6ce77,2019-03-05 15:07:03,0.3,0.0,0.0,0.0,247.0,1,1.0,247.0,0.0
1209947,fffff6ce77,2019-03-06 16:26:35,0.1,0.0,0.0,0.0,109.0,2,2.0,110.0,0.0
1209948,fffff6ce77,2019-03-06 16:53:10,0.2,0.0,0.0,0.0,245.0,1,1.0,245.0,0.0


In [5]:
train_purch = train_purch\
    .groupby(['client_id','transaction_datetime'],
            as_index=False)\
    .agg({'regular_points_received': 'mean',
          'express_points_received': 'mean',
          'regular_points_spent': 'mean', 
          'express_points_spent': 'mean', 
          'purchase_sum': 'mean',
         'product_id': 'count',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
train_purch.rename(columns = {'product_id': 'n_products'}, inplace=True)
train_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,19,21.0,1007.0,0.0
1,000012768d,2018-12-16 08:56:01,5.7,0.0,0.0,0.0,574.0,11,14.0,575.0,0.0
2,000012768d,2019-03-08 10:12:03,8.0,0.0,0.0,0.0,803.0,16,13.0,804.0,0.0
3,000012768d,2019-03-14 15:01:47,2.0,0.0,0.0,0.0,419.0,6,6.0,419.0,0.0
4,000036f903,2018-11-28 10:48:36,1.2,0.0,0.0,0.0,241.0,5,4.0,241.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2814986,fffeca6d22,2019-02-22 09:34:13,5.0,0.0,0.0,0.0,110.0,3,3.0,111.0,0.0
2814987,fffeca6d22,2019-02-22 09:34:53,6.0,0.0,0.0,0.0,138.0,1,2.0,139.0,0.0
2814988,fffeca6d22,2019-02-28 10:25:24,0.6,0.0,0.0,0.0,128.0,2,3.0,128.0,0.0
2814989,fffeca6d22,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,3,3.0,100.0,0.0


In [6]:
train_purch.transaction_datetime.min()

Timestamp('2018-11-21 21:02:33')

In [7]:
train_purch.transaction_datetime.max()

Timestamp('2019-03-18 23:19:28')

In [8]:
test_purch.transaction_datetime.min()

Timestamp('2018-11-21 21:02:51')

In [9]:
test_purch.transaction_datetime.max()

Timestamp('2019-03-18 22:08:09')

In [10]:
start = min(
    train_purch.transaction_datetime.min(),
    test_purch.transaction_datetime.min()
)
start

Timestamp('2018-11-21 21:02:33')

In [11]:
finish = max(
    train_purch.transaction_datetime.max(),
    test_purch.transaction_datetime.max()
)
finish

Timestamp('2019-03-18 23:19:28')

Number of intervals:

In [12]:
n_intervals = 8

In [13]:
bins = [start + i * (finish - start) / n_intervals for i in range(0, n_intervals + 1)]
bins

[Timestamp('2018-11-21 21:02:33'),
 Timestamp('2018-12-06 12:19:39.875000'),
 Timestamp('2018-12-21 03:36:46.750000'),
 Timestamp('2019-01-04 18:53:53.625000'),
 Timestamp('2019-01-19 10:11:00.500000'),
 Timestamp('2019-02-03 01:28:07.375000'),
 Timestamp('2019-02-17 16:45:14.250000'),
 Timestamp('2019-03-04 08:02:21.125000'),
 Timestamp('2019-03-18 23:19:28')]

In [14]:
test_purch['interval'] = pd.cut(test_purch.transaction_datetime, bins, labels=False)
test_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red,interval
0,00010925a5,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,5,7.0,303.0,0.0,0
1,00010925a5,2018-12-01 09:17:18,0.8,0.0,0.0,0.0,132.0,3,2.0,132.0,0.0,0
2,00010925a5,2018-12-03 07:52:46,0.7,0.0,0.0,0.0,149.0,3,3.0,150.0,0.0,0
3,00010925a5,2018-12-06 09:59:19,1.7,0.0,0.0,0.0,349.0,6,7.0,349.0,0.0,0
4,00010925a5,2018-12-17 12:51:26,5.8,0.0,0.0,0.0,581.0,8,8.0,582.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1209945,fffff6ce77,2019-02-28 12:56:31,1.8,0.0,0.0,0.0,446.0,9,10.0,446.0,0.0,6
1209946,fffff6ce77,2019-03-05 15:07:03,0.3,0.0,0.0,0.0,247.0,1,1.0,247.0,0.0,7
1209947,fffff6ce77,2019-03-06 16:26:35,0.1,0.0,0.0,0.0,109.0,2,2.0,110.0,0.0,7
1209948,fffff6ce77,2019-03-06 16:53:10,0.2,0.0,0.0,0.0,245.0,1,1.0,245.0,0.0,7


In [15]:
train_purch['interval'] = pd.cut(train_purch.transaction_datetime, bins, labels=False)
train_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red,interval
0,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,19,21.0,1007.0,0.0,0.0
1,000012768d,2018-12-16 08:56:01,5.7,0.0,0.0,0.0,574.0,11,14.0,575.0,0.0,1.0
2,000012768d,2019-03-08 10:12:03,8.0,0.0,0.0,0.0,803.0,16,13.0,804.0,0.0,7.0
3,000012768d,2019-03-14 15:01:47,2.0,0.0,0.0,0.0,419.0,6,6.0,419.0,0.0,7.0
4,000036f903,2018-11-28 10:48:36,1.2,0.0,0.0,0.0,241.0,5,4.0,241.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2814986,fffeca6d22,2019-02-22 09:34:13,5.0,0.0,0.0,0.0,110.0,3,3.0,111.0,0.0,6.0
2814987,fffeca6d22,2019-02-22 09:34:53,6.0,0.0,0.0,0.0,138.0,1,2.0,139.0,0.0,6.0
2814988,fffeca6d22,2019-02-28 10:25:24,0.6,0.0,0.0,0.0,128.0,2,3.0,128.0,0.0,6.0
2814989,fffeca6d22,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,3,3.0,100.0,0.0,7.0


In [16]:
test_purch.isna().sum()

client_id                  0
transaction_datetime       0
regular_points_received    0
express_points_received    0
regular_points_spent       0
express_points_spent       0
purchase_sum               0
n_products                 0
product_quantity           0
trn_sum_from_iss           0
trn_sum_from_red           0
interval                   0
dtype: int64

In [17]:
train_purch.isna().sum()

client_id                  0
transaction_datetime       0
regular_points_received    0
express_points_received    0
regular_points_spent       0
express_points_spent       0
purchase_sum               0
n_products                 0
product_quantity           0
trn_sum_from_iss           0
trn_sum_from_red           0
interval                   1
dtype: int64

In [18]:
train_purch.loc[train_purch.interval.isna()]

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red,interval
434367,27df8bc8bb,2018-11-21 21:02:33,1.5,0.0,0.0,0.0,224.53,5,8.0,226.0,0.0,NaN


In [19]:
train_purch.fillna(value=0, inplace=True)

In [20]:
train_purch.isna().sum()

client_id                  0
transaction_datetime       0
regular_points_received    0
express_points_received    0
regular_points_spent       0
express_points_spent       0
purchase_sum               0
n_products                 0
product_quantity           0
trn_sum_from_iss           0
trn_sum_from_red           0
interval                   0
dtype: int64

In [21]:
train_purch.dtypes

client_id                          object
transaction_datetime       datetime64[ns]
regular_points_received           float64
express_points_received           float64
regular_points_spent              float64
express_points_spent              float64
purchase_sum                      float64
n_products                          int64
product_quantity                  float64
trn_sum_from_iss                  float64
trn_sum_from_red                  float64
interval                          float64
dtype: object

In [22]:
train_purch.interval = train_purch.interval.astype('int')

In [23]:
train_purch.dtypes

client_id                          object
transaction_datetime       datetime64[ns]
regular_points_received           float64
express_points_received           float64
regular_points_spent              float64
express_points_spent              float64
purchase_sum                      float64
n_products                          int64
product_quantity                  float64
trn_sum_from_iss                  float64
trn_sum_from_red                  float64
interval                            int64
dtype: object

In [24]:
test_purch = test_purch\
    .groupby(['client_id','interval'])\
    .agg({'regular_points_received': 'sum',
          'express_points_received': 'sum',
          'regular_points_spent': 'sum', 
          'express_points_spent': 'sum', 
          'purchase_sum': 'sum',
         'n_products': 'sum',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
test_purch

regular_points_received  express_points_received  \
client_id  interval                                                     
00010925a5 0                             4.7                      0.0   
           1                             6.7                      0.0   
           2                             2.1                      0.0   
           4                             2.3                      0.0   
           5                             2.1                      0.0   
...                                      ...                      ...   
fffff6ce77 3                            54.0                      0.0   
           4                            60.3                      0.0   
           5                            10.2                      0.0   
           6                            20.0                      0.0   
           7                             7.3                      0.0   

                     regular_points_spent  express_points_spent  purchase_sum  \
client_id  interval                                                             
00010925a5 0                          0.0                   0.0         933.0   
           1                          0.0                   0.0         764.0   
           2                        -17.0                   0.0         438.0   
           4                          0.0                   0.0         463.0   
           5                          0.0                   0.0         427.0   
...                                   ...                   ...           ...   
fffff6ce77 3                          0.0                   0.0        5854.0   
           4                          0.0                   0.0        2397.0   
           5                       -302.0                   0.0        1795.0   
           6                          0.0                   0.0        3156.0   
           7                          0.0                   0.0        1277.0   

                     n_products  product_quantity  trn_sum_from_iss  \
client_id  interval                                                   
00010925a5 0                 17              19.0             934.0   
           1                 11              11.0             766.0   
           2                  5               4.0             420.0   
           4                  5               5.0             463.0   
           5                  7               7.0             429.0   
...                         ...               ...               ...   
fffff6ce77 3                 55              81.0            5857.0   
           4                 34              40.0            2403.0   
           5                 34              37.0            1497.0   
           6                 31              40.0            3160.0   
           7                 13              13.0            1278.0   

                     trn_sum_from_red  
client_id  interval                    
00010925a5 0                      0.0  
           1                      0.0  
           2                    438.0  
           4                      0.0  
           5                      0.0  
...                               ...  
fffff6ce77 3                      0.0  
           4                      0.0  
           5                   1055.0  
           6                      0.0  
           7                      0.0  

[356982 rows x 9 columns]

In [25]:
train_purch = train_purch\
    .groupby(['client_id','interval'])\
    .agg({'regular_points_received': 'sum',
          'express_points_received': 'sum',
          'regular_points_spent': 'sum', 
          'express_points_spent': 'sum', 
          'purchase_sum': 'sum',
         'n_products': 'sum',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
train_purch

regular_points_received  express_points_received  \
client_id  interval                                                     
000012768d 0                            10.0                      0.0   
           1                             5.7                      0.0   
           7                            10.0                      0.0   
000036f903 0                             5.0                     60.0   
           1                             4.4                      0.0   
...                                      ...                      ...   
fffeca6d22 3                             0.4                      0.0   
           4                             1.2                      0.0   
           5                             0.4                      0.0   
           6                            33.3                      0.0   
           7                             1.2                      0.0   

                     regular_points_spent  express_points_spent  purchase_sum  \
client_id  interval                                                             
000012768d 0                          0.0                   0.0       1007.00   
           1                          0.0                   0.0        574.00   
           7                          0.0                   0.0       1222.00   
000036f903 0                          0.0                   0.0       1027.00   
           1                          0.0                   0.0        930.00   
...                                   ...                   ...           ...   
fffeca6d22 3                          0.0                   0.0        118.00   
           4                          0.0                   0.0        265.00   
           5                          0.0                   0.0         97.00   
           6                          0.0                   0.0        987.86   
           7                          0.0                   0.0        264.00   

                     n_products  product_quantity  trn_sum_from_iss  \
client_id  interval                                                   
000012768d 0                 19              21.0            1007.0   
           1                 11              14.0             575.0   
           7                 22              19.0            1223.0   
000036f903 0                 19              18.0            1027.0   
           1                 17              17.0             931.0   
...                         ...               ...               ...   
fffeca6d22 3                  2               3.0             120.0   
           4                  6               8.0             267.0   
           5                  2               2.0              98.0   
           6                 24              28.0             992.0   
           7                  6               7.0             266.0   

                     trn_sum_from_red  
client_id  interval                    
000012768d 0                      0.0  
           1                      0.0  
           7                      0.0  
000036f903 0                      0.0  
           1                      0.0  
...                               ...  
fffeca6d22 3                      0.0  
           4                      0.0  
           5                      0.0  
           6                      0.0  
           7                      0.0  

[833170 rows x 9 columns]

In [26]:
scaler_2 = StandardScaler().fit(train_purch)

In [27]:
scaled_train = scaler_2.transform(train_purch)

In [28]:
train_purch = pd.DataFrame(scaled_train,
                           columns=train_purch.columns,
                           index=train_purch.index)
train_purch.reset_index(inplace=True)
train_purch

,client_id,interval,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,0,-0.147120,-0.041893,0.254342,0.180536,-0.277224,-0.011427,-0.119189,-0.262736,-0.288309
1,000012768d,1,-0.352561,-0.041893,0.254342,0.180536,-0.552725,-0.462854,-0.404303,-0.544941,-0.288309
2,000012768d,7,-0.147120,-0.041893,0.254342,0.180536,-0.140427,0.157858,-0.200650,-0.121633,-0.288309
3,000036f903,0,-0.386005,18.443831,0.254342,0.180536,-0.264498,-0.011427,-0.241381,-0.249671,-0.288309
4,000036f903,1,-0.414671,-0.041893,0.254342,0.180536,-0.326216,-0.124284,-0.282111,-0.312383,-0.288309
...,...,...,...,...,...,...,...,...,...,...,...
833165,fffeca6d22,3,-0.605778,-0.041893,0.254342,0.180536,-0.842861,-0.970709,-0.852338,-0.842172,-0.288309
833166,fffeca6d22,4,-0.567557,-0.041893,0.254342,0.180536,-0.749331,-0.744996,-0.648686,-0.746144,-0.288309
833167,fffeca6d22,5,-0.605778,-0.041893,0.254342,0.180536,-0.856223,-0.970709,-0.893069,-0.856544,-0.288309
833168,fffeca6d22,6,0.966082,-0.041893,0.254342,0.180536,-0.289402,0.270715,0.165924,-0.272534,-0.288309


In [29]:
scaled_test = scaler_2.transform(test_purch)

In [30]:
test_purch = pd.DataFrame(scaled_test,
                           columns=test_purch.columns,
                           index=test_purch.index)
test_purch.reset_index(inplace=True)
test_purch

,client_id,interval,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,0,-0.400338,-0.041893,0.254342,0.180536,-0.324307,-0.124284,-0.200650,-0.310423,-0.288309
1,00010925a5,1,-0.304784,-0.041893,0.254342,0.180536,-0.431836,-0.462854,-0.526494,-0.420170,-0.288309
2,00010925a5,2,-0.524558,-0.041893,-0.097441,0.180536,-0.639257,-0.801424,-0.811608,-0.646196,1.011890
3,00010925a5,4,-0.515002,-0.041893,0.254342,0.180536,-0.623351,-0.801424,-0.770877,-0.618106,-0.288309
4,00010925a5,5,-0.524558,-0.041893,0.254342,0.180536,-0.646256,-0.688568,-0.689416,-0.640317,-0.288309
...,...,...,...,...,...,...,...,...,...,...,...
356977,fffff6ce77,3,1.955064,-0.041893,0.254342,0.180536,2.806744,2.019994,2.324640,2.905547,-0.288309
356978,fffff6ce77,4,2.256059,-0.041893,0.254342,0.180536,0.607182,0.834998,0.654690,0.649207,-0.288309
356979,fffff6ce77,5,-0.137565,-0.041893,-5.994970,0.180536,0.224152,0.834998,0.532499,0.057359,2.843448
356980,fffff6ce77,6,0.330649,-0.041893,0.254342,0.180536,1.090106,0.665713,0.654690,1.143721,-0.288309


In [31]:
test_purch = test_purch.pivot(index='client_id', columns='interval')
test_purch.fillna(0, inplace=True)
test_purch

regular_points_received                                          \
interval                         0         1         2         3         4   
client_id                                                                    
00010925a5               -0.400338 -0.304784 -0.524558  0.000000 -0.515002   
00035a21d9                0.000000  0.000000  0.000000  0.000000  0.000000   
00038f9200                0.000000  0.211207 -0.061121 -0.400338 -0.386005   
0004315e57               -0.366894  0.000000  0.187318  0.000000 -0.309561   
0006fca4bf                0.000000  0.000000  0.000000  0.000000  0.000000   
...                            ...       ...       ...       ...       ...   
fff9aa288c               -0.147120  0.000000  0.000000 -0.553224  0.430981   
fff9f772f6                0.000000  0.039210  0.000000 -0.257007  0.000000   
fffe0ed719               -0.543668 -0.429004 -0.562779 -0.558001 -0.586668   
fffea1204c               -0.515002 -0.586668 -0.591445 -0.505447 -0.491114   
fffff6ce77                0.292428  0.497868  0.655532  1.955064  2.256059   

                                         express_points_received            \
interval           5         6         7                       0         1   
client_id                                                                    
00010925a5 -0.524558 -0.295228 -0.290451               -0.041893 -0.041893   
00035a21d9 -0.204452  0.082209  0.000000                0.000000  0.000000   
00038f9200 -0.371671 -0.233118  0.822751                0.000000 -0.041893   
0004315e57 -0.132787 -0.237896  0.349760               -0.041893  0.000000   
0006fca4bf  0.000000 -0.577112  1.018637                0.000000  0.000000   
...              ...       ...       ...                     ...       ...   
fff9aa288c -0.252229  0.000000 -0.591445               -0.041893  0.000000   
fff9f772f6  0.000000 -0.276118  2.576164                0.000000 -0.041893   
fffe0ed719 -0.586668  0.908750  0.698531               -0.041893 -0.041893   
fffea1204c -0.333450 -0.108898 -0.620111               -0.041893 -0.041893   
fffff6ce77 -0.137565  0.330649 -0.276118               -0.041893 -0.041893   

            ... trn_sum_from_iss           trn_sum_from_red            \
interval    ...                6         7                0         1   
client_id   ...                                                         
00010925a5  ...        -0.000127  0.027963        -0.288309 -0.288309   
00035a21d9  ...        -0.430622  0.000000         0.000000  0.000000   
00038f9200  ...         0.203688 -0.222887         0.000000 -0.288309   
0004315e57  ...        -0.264042  0.390519        -0.288309  0.000000   
0006fca4bf  ...        -0.781420  0.339565         0.000000  0.000000   
...         ...              ...       ...              ...       ...   
fff9aa288c  ...         0.000000 -0.825188        -0.288309  0.000000   
fff9f772f6  ...        -0.440421  1.268492         0.000000 -0.288309   
fffe0ed719  ...         1.469041  2.281690        -0.288309 -0.288309   
fffea1204c  ...         0.171678 -0.897046        -0.288309 -0.288309   
fffff6ce77  ...         1.143721 -0.085704        -0.288309 -0.288309   

                                                                        
interval           2         3         4         5         6         7  
client_id                                                               
00010925a5  1.011890  0.000000 -0.288309 -0.288309 -0.288309 -0.288309  
00035a21d9  0.000000  0.000000  0.000000 -0.288309  2.047893  0.000000  
00038f9200 -0.288309  0.726915 -0.288309 -0.288309 -0.288309 -0.288309  
0004315e57 -0.288309  0.000000  1.831193 -0.288309  1.828225 -0.288309  
0006fca4bf  0.000000  0.000000  0.000000  0.000000 -0.288309 -0.288309  
...              ...       ...       ...       ...       ...       ...  
fff9aa288c  0.000000 -0.288309 -0.288309 -0.288309  0.000000  0.281641  
fff9f772f6  0.000000 -0.288309  0.000000  0.000000 -0.288309 -0.288309  

In [32]:
train_purch = train_purch.pivot(index='client_id', columns='interval')
train_purch.fillna(0, inplace=True)
train_purch

regular_points_received                                          \
interval                         0         1         2         3         4   
client_id                                                                    
000012768d               -0.147120 -0.352561  0.000000  0.000000  0.000000   
000036f903               -0.386005 -0.414671  0.077432 -0.429004 -0.419448   
0001f552b0                0.163430  0.000000 -0.065899  0.000000 -0.553224   
00020e7b18                3.168598  0.000000  1.501184  1.458184  0.053543   
000220a0a7                0.000000  0.091765  1.634959  0.000000  0.058321   
...                            ...       ...       ...       ...       ...   
fffd5cd0c6               -0.567557 -0.376449 -0.605778 -0.443337 -0.061121   
fffd63dfe3                0.000000  0.000000  0.000000 -0.567557 -0.448115   
fffd8c9d7d               -0.601001  0.000000  0.000000  0.000000 -0.343005   
fffe0abb97               -0.328672  0.000000 -0.572334  0.000000 -0.295228   
fffeca6d22               -0.209230 -0.519780  0.000000 -0.605778 -0.567557   

                                         express_points_received            \
interval           5         6         7                       0         1   
client_id                                                                    
000012768d  0.000000  0.000000 -0.147120               -0.041893 -0.041893   
000036f903 -0.209230 -0.166231 -0.429004               18.443831 -0.041893   
0001f552b0 -0.519780 -0.515002  1.510739               -0.041893  0.000000   
00020e7b18  1.572849  1.419963  0.120431               -0.041893  0.000000   
000220a0a7 -0.558001 -0.003789  0.000988                0.000000 -0.041893   
...              ...       ...       ...                     ...       ...   
fffd5cd0c6  0.000000 -0.314339 -0.328672               -0.041893 -0.041893   
fffd63dfe3  0.000000  0.000000 -0.548446                0.000000  0.000000   
fffd8c9d7d  0.000000 -0.467225 -0.562779               -0.041893  0.000000   
fffe0abb97 -0.591445 -0.276118 -0.624889               -0.041893  0.000000   
fffeca6d22 -0.605778  0.966082 -0.567557               -0.041893 -0.041893   

            ... trn_sum_from_iss           trn_sum_from_red            \
interval    ...                6         7                0         1   
client_id   ...                                                         
000012768d  ...         0.000000 -0.121633        -0.288309 -0.288309   
000036f903  ...         0.350670 -0.372482        -0.288309 -0.288309   
0001f552b0  ...        -0.595895  0.199768        -0.288309  0.000000   
00020e7b18  ...         1.977926  0.109619        -0.288309  0.000000   
000220a0a7  ...         0.118112  0.279465         0.000000 -0.288309   
...         ...              ...       ...              ...       ...   
fffd5cd0c6  ...        -0.079171 -0.073292        -0.288309 -0.288309   
fffd63dfe3  ...         0.000000 -0.704989         0.000000  0.000000   
fffd8c9d7d  ...        -0.482882 -0.744184        -0.288309  0.000000   
fffe0abb97  ...        -0.441074 -0.912070         0.008540  0.000000   
fffeca6d22  ...        -0.272534 -0.746797        -0.288309 -0.288309   

                                                                        
interval           2         3         4         5         6         7  
client_id                                                               
000012768d  0.000000  0.000000  0.000000  0.000000  0.000000 -0.288309  
000036f903 -0.288309 -0.288309 -0.288309 -0.288309 -0.288309 -0.288309  
0001f552b0 -0.288309  0.000000 -0.288309 -0.288309 -0.288309 -0.288309  
00020e7b18 -0.288309 -0.288309  5.470562  5.146996  5.719915 -0.021145  
000220a0a7 -0.288309  0.000000 -0.288309 -0.288309 -0.288309 -0.288309  
...              ...       ...       ...       ...       ...       ...  
fffd5cd0c6 -0.288309 -0.288309 -0.288309  0.000000 -0.288309 -0.288309  
fffd63dfe3  0.000000 -0.288309 -0.288309  0.000000  0.000000 -0.288309  

In [33]:
columns = train_purch.columns.to_flat_index().to_list()
columns = [col[0] + '_' + str(col[1]) for col in columns]

In [34]:
train_purch.columns = columns
train_purch

,regular_points_received_0,regular_points_received_1,regular_points_received_2,regular_points_received_3,regular_points_received_4,regular_points_received_5,regular_points_received_6,regular_points_received_7,express_points_received_0,express_points_received_1,...,trn_sum_from_iss_6,trn_sum_from_iss_7,trn_sum_from_red_0,trn_sum_from_red_1,trn_sum_from_red_2,trn_sum_from_red_3,trn_sum_from_red_4,trn_sum_from_red_5,trn_sum_from_red_6,trn_sum_from_red_7
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,-0.147120,-0.352561,0.000000,0.000000,0.000000,0.000000,0.000000,-0.147120,-0.041893,-0.041893,...,0.000000,-0.121633,-0.288309,-0.288309,0.000000,0.000000,0.000000,0.000000,0.000000,-0.288309
000036f903,-0.386005,-0.414671,0.077432,-0.429004,-0.419448,-0.209230,-0.166231,-0.429004,18.443831,-0.041893,...,0.350670,-0.372482,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309
0001f552b0,0.163430,0.000000,-0.065899,0.000000,-0.553224,-0.519780,-0.515002,1.510739,-0.041893,0.000000,...,-0.595895,0.199768,-0.288309,0.000000,-0.288309,0.000000,-0.288309,-0.288309,-0.288309,-0.288309
00020e7b18,3.168598,0.000000,1.501184,1.458184,0.053543,1.572849,1.419963,0.120431,-0.041893,0.000000,...,1.977926,0.109619,-0.288309,0.000000,-0.288309,-0.288309,5.470562,5.146996,5.719915,-0.021145
000220a0a7,0.000000,0.091765,1.634959,0.000000,0.058321,-0.558001,-0.003789,0.000988,0.000000,-0.041893,...,0.118112,0.279465,0.000000,-0.288309,-0.288309,0.000000,-0.288309,-0.288309,-0.288309,-0.288309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffd5cd0c6,-0.567557,-0.376449,-0.605778,-0.443337,-0.061121,0.000000,-0.314339,-0.328672,-0.041893,-0.041893,...,-0.079171,-0.073292,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,0.000000,-0.288309,-0.288309
fffd63dfe3,0.000000,0.000000,0.000000,-0.567557,-0.448115,0.000000,0.000000,-0.548446,0.000000,0.000000,...,0.000000,-0.704989,0.000000,0.000000,0.000000,-0.288309,-0.288309,0.000000,0.000000,-0.288309
fffd8c9d7d,-0.601001,0.000000,0.000000,0.000000,-0.343005,0.000000,-0.467225,-0.562779,-0.041893,0.000000,...,-0.482882,-0.744184,-0.288309,0.000000,0.000000,0.000000,-0.288309,0.000000,-0.288309,-0.288309


In [35]:
test_purch.columns = columns
test_purch

,regular_points_received_0,regular_points_received_1,regular_points_received_2,regular_points_received_3,regular_points_received_4,regular_points_received_5,regular_points_received_6,regular_points_received_7,express_points_received_0,express_points_received_1,...,trn_sum_from_iss_6,trn_sum_from_iss_7,trn_sum_from_red_0,trn_sum_from_red_1,trn_sum_from_red_2,trn_sum_from_red_3,trn_sum_from_red_4,trn_sum_from_red_5,trn_sum_from_red_6,trn_sum_from_red_7
client_id,,,,,,,,,,,,,,,,,,,,,
00010925a5,-0.400338,-0.304784,-0.524558,0.000000,-0.515002,-0.524558,-0.295228,-0.290451,-0.041893,-0.041893,...,-0.000127,0.027963,-0.288309,-0.288309,1.011890,0.000000,-0.288309,-0.288309,-0.288309,-0.288309
00035a21d9,0.000000,0.000000,0.000000,0.000000,0.000000,-0.204452,0.082209,0.000000,0.000000,0.000000,...,-0.430622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.288309,2.047893,0.000000
00038f9200,0.000000,0.211207,-0.061121,-0.400338,-0.386005,-0.371671,-0.233118,0.822751,0.000000,-0.041893,...,0.203688,-0.222887,0.000000,-0.288309,-0.288309,0.726915,-0.288309,-0.288309,-0.288309,-0.288309
0004315e57,-0.366894,0.000000,0.187318,0.000000,-0.309561,-0.132787,-0.237896,0.349760,-0.041893,0.000000,...,-0.264042,0.390519,-0.288309,0.000000,-0.288309,0.000000,1.831193,-0.288309,1.828225,-0.288309
0006fca4bf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.577112,1.018637,0.000000,0.000000,...,-0.781420,0.339565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.288309,-0.288309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff9aa288c,-0.147120,0.000000,0.000000,-0.553224,0.430981,-0.252229,0.000000,-0.591445,-0.041893,0.000000,...,0.000000,-0.825188,-0.288309,0.000000,0.000000,-0.288309,-0.288309,-0.288309,0.000000,0.281641
fff9f772f6,0.000000,0.039210,0.000000,-0.257007,0.000000,0.000000,-0.276118,2.576164,0.000000,-0.041893,...,-0.440421,1.268492,0.000000,-0.288309,0.000000,-0.288309,0.000000,0.000000,-0.288309,-0.288309
fffe0ed719,-0.543668,-0.429004,-0.562779,-0.558001,-0.586668,-0.586668,0.908750,0.698531,-0.041893,-0.041893,...,1.469041,2.281690,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309,-0.288309


In [36]:
test_purch.to_csv('test_purch_4_periods.csv')

In [37]:
train_purch.to_csv('train_purch_4_periods.csv')